In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import re
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.service import Service

In [2]:
# Path to the downloaded ChromeDriver
driver_path = "C:/chromedriver/chromedriver.exe"  # Update this path

# Create a new instance of the Chrome driver
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Test if it's working by opening a web page
driver.get("https://www.google.com")

# Close the driver
driver.quit()


In [3]:
'''Products Aldi-Sued Script'''

Lists = ["https://www.aldi-sued.de/de/produkte/produktsortiment/kuehlung-und-tiefkuehlkost.html",
        "https://www.aldi-sued.de/de/produkte/produktsortiment/nahrungsmittel.html",
        "https://www.aldi-sued.de/de/produkte/produktsortiment/brot-aufstrich-und-cerealien.html",
        "https://www.aldi-sued.de/de/produkte/produktsortiment/kaffee-und-tee.html",
        "https://www.aldi-sued.de/de/produkte/produktsortiment/getraenke.html",
        "https://www.aldi-sued.de/de/produkte/produktsortiment/suessigkeiten-und-snacks.html",
        "https://www.aldi-sued.de/de/produkte/produktsortiment/drogerie-und-kosmetik.html",
        "https://www.aldi-sued.de/de/produkte/ernaehrungsformen/bio.html",
        "https://www.aldi-sued.de/de/produkte/ernaehrungsformen/fairtrade.html",
        "https://www.aldi-sued.de/de/produkte/ernaehrungsformen/vegan.html",
        "https://www.aldi-sued.de/de/produkte/ernaehrungsformen/vegetarisch.html",
        "https://www.aldi-sued.de/de/produkte/produktsortiment/baby-und-kind.html"]

In [29]:
# One category at a time
main_url = Lists[-12]
print('Main Website Page', main_url)

def request_to_AldiSued(url):
    driver_path = "C:/chromedriver/chromedriver.exe"  
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)

    driver.get(url)  # Load the page
    time.sleep(10)
    return driver
 
def showMore_dynamic(driver):
    condition = True
    count = 1
    while condition:
        try:
            btn = driver.find_element_by_id("showMore")   
            btn.click()
            time.sleep(5)
            print ('          Number of Times Show More button clicked :', count)
            count += 1
        except Exception as e:
            print("Exception: There is no more show button")
            condition = False
    html_string = driver.page_source
    soup = BeautifulSoup(html_string, "html.parser")
    return soup #html_string
driver = request_to_AldiSued(main_url)
input_alert = input('Note: To dynamically operate the website. It is manually required to accept the cookies and delete the chatbot element.')
html_string = showMore_dynamic(driver)


Main Website Page https://www.aldi-sued.de/de/produkte/produktsortiment/kuehlung-und-tiefkuehlkost.html
Note: To dynamically operate the website. It is manually required to accept the cookies and delete the chatbot element.


C:\Users\AK traders\AppData\Local\Temp\ipykernel_18784\3077468239.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_id("showMore")


          Number of Times Show More button clicked : 1
          Number of Times Show More button clicked : 2
          Number of Times Show More button clicked : 3
          Number of Times Show More button clicked : 4
          Number of Times Show More button clicked : 5
          Number of Times Show More button clicked : 6
          Number of Times Show More button clicked : 7
          Number of Times Show More button clicked : 8
          Number of Times Show More button clicked : 9
          Number of Times Show More button clicked : 10
          Number of Times Show More button clicked : 11
          Number of Times Show More button clicked : 12
          Number of Times Show More button clicked : 13
          Number of Times Show More button clicked : 14
          Number of Times Show More button clicked : 15
          Number of Times Show More button clicked : 16
          Number of Times Show More button clicked : 17
          Number of Times Show More button clicked : 18
 

In [30]:


def request_product_link(url):
    print (url)
    first_response = requests.get(url)
    if first_response.status_code == 200:
        html_content = first_response.text
    else:
        print("Error:", first_response.status_code)
    soup = BeautifulSoup(html_content, "html.parser")
    return soup
def extract_information(product_soup):
    name = product_soup.find('h1', class_='at-productname_lbl').text
    price = product_soup.find('span',class_='pdp_price__now').text
    description = product_soup.find('div',class_='infobox').text
    extra_images = product_soup.find_all('div', {'class':'badges-gallery__item'})
    label_images = []
    for extra_image in extra_images:
        label_images.append(extra_image.find('img').get('data-src'))
    left_side_images = product_soup.find_all('div',{'class':'topLeft'})
    left_images = []
    for left_image in left_side_images:
        left_images.append(left_image.find('img').get('data-src'))

    return name, price, description, label_images, left_images

def beautifulsoup(html_string):
    soup = html_string
   # soup = BeautifulSoup(html_string, "html.parser")
    category = soup.find('h1', {'class': 'plp_title'}).text
    links_images = soup.find_all('article',{'class':'wrapper'})
    data_to_save = {}
    count = 0
    for link_image in links_images:
        count += 1
        try:
            
            if count >= 0:
                url = "https://www.aldi-sued.de"+str(link_image.find('a').get('href'))

                data_to_save['Category'] = category
                data_to_save['Product_link'] = "https://www.aldi-sued.de"+str(link_image.find('a').get('href'))
                data_to_save['Product_image'] = str(link_image.find('img').get('data-src'))
                product_soup = request_product_link(url)
                name, price, description, label_images, left_images = extract_information(product_soup)
                data_to_save['Name'] = name
                data_to_save['Price'] = price
                data_to_save['Description'] = description
                all_label_images = label_images + left_images
                data_to_save['Label_images'] = all_label_images
                time.sleep(2)
                final_data = save_json(data_to_save)
                print ('Number of Data Point Extracted: ', count)

            else:
                #break
                continue
        except:
            print ('Some Error at number :', count)
            continue
        

def save_json(final_data):
    try:
        with open('AldiSued.json', 'r') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        existing_data = []

    existing_data.append(final_data)

    with open('AldiSued.json', 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

    return None
 
if __name__ == "__main__":
  #  driver = request_to_AldiSued(main_url)
  #  input_alert = input('Note: To dynamically operate the website. It is manually required to accept the cookies and delete the chatbot element.')
  #  html_string = showMore_dynamic(driver)
    beautifulsoup(html_string)

https://www.aldi-sued.de/de/p.fair--gut-tierwohlmilch--l.490000000000715707.html
Number of Data Point Extracted:  1
https://www.aldi-sued.de/de/p.frankenland-vollmilch---fett--l.490000000000012013.html
Number of Data Point Extracted:  2
https://www.aldi-sued.de/de/p.nafa-kartoffelsalat--g.490200000000723332.html
Number of Data Point Extracted:  3
https://www.aldi-sued.de/de/p.nafa-kartoffelsalat--g.490100000000723332.html
Number of Data Point Extracted:  4
https://www.aldi-sued.de/de/p.zott-monte-maxi--g.490000000000051937.html
Number of Data Point Extracted:  5
https://www.aldi-sued.de/de/p.gefullter-brie-kraeuter--g.490000000000700846.html
Number of Data Point Extracted:  6
https://www.aldi-sued.de/de/p.cucina-nobile-bueffelmozzarella-minis--g.490000000000709531.html
Number of Data Point Extracted:  7
https://www.aldi-sued.de/de/p.nafa-feinkostsalat--g.490000000000723271.html
Number of Data Point Extracted:  8
https://www.aldi-sued.de/de/p.fair--gut-tierwohlmilch--l-.4900000000007157

Number of Data Point Extracted:  68
https://www.aldi-sued.de/de/p.meine-metzgerei-haehnchen-brustfilet-teilstueck--g.490000000000044011.html
Number of Data Point Extracted:  69
https://www.aldi-sued.de/de/p.gourmet-finest-cuisine-dessert-variation--g.490000000000012912.html
Number of Data Point Extracted:  70
https://www.aldi-sued.de/de/p.gourmet-finest-cuisine-zanderfilets--g.490000000000013049.html
Number of Data Point Extracted:  71
https://www.aldi-sued.de/de/p.gut-bio-bio-obst--g.490000000000815010.html
Number of Data Point Extracted:  72
https://www.aldi-sued.de/de/p.wonnemeyer-salatdressing--ml.490000000000002589.html
Number of Data Point Extracted:  73
https://www.aldi-sued.de/de/p.all-seasons-rahmspinat--g.490000000000003389.html
Number of Data Point Extracted:  74
https://www.aldi-sued.de/de/p.milsani-frucht-auf-joghurt--g.490000000000005800.html
Number of Data Point Extracted:  75
https://www.aldi-sued.de/de/p.meine-kaesetheke-kuestenkaese--tilsiter-g.490100000000048531.html

Number of Data Point Extracted:  134
https://www.aldi-sued.de/de/p.hofburger-back-camembert-mit-dip--g.490000000000012369.html
Number of Data Point Extracted:  135
https://www.aldi-sued.de/de/p.meine-kaesetheke-schweizer-kaesespezialitaet--g.490000000000009094.html
Number of Data Point Extracted:  136
https://www.aldi-sued.de/de/p.roi-de-trefle-franzoesischer-weichkaese--g.490000000000006655.html
Number of Data Point Extracted:  137
https://www.aldi-sued.de/de/p.hofburger-raeucherscheiben--g.490000000000045867.html
Number of Data Point Extracted:  138
https://www.aldi-sued.de/de/p.roi-de-trefle-zarte-scheiben--g.490000000000044455.html
Number of Data Point Extracted:  139
https://www.aldi-sued.de/de/p.jacks-farm-cevapcici--kg.490000000000045395.html
Number of Data Point Extracted:  140
https://www.aldi-sued.de/de/p.gut-drei-eichen-kasseler-aufschnitt--g.490000000000052024.html
Number of Data Point Extracted:  141
https://www.aldi-sued.de/de/p.wonnemeyer-feinkostsalat--g.490200000000039

Number of Data Point Extracted:  197
https://www.aldi-sued.de/de/p.gut-drei-eichen-schinkenbockwurst--g.000000000000203228.html
Number of Data Point Extracted:  198
https://www.aldi-sued.de/de/p.gut-bio-bio-chicken-wings--g.000000000000213251.html
Number of Data Point Extracted:  199
https://www.aldi-sued.de/de/p.gut-bio-bio-landhendl--kg.000000000000319005.html
Number of Data Point Extracted:  200
https://www.aldi-sued.de/de/p.gut-bio-bio-mini-berner-wuerstchen--g.000000000000481479.html
Number of Data Point Extracted:  201
https://www.aldi-sued.de/de/p.gut-bio-bio-kaesestuecke--g-gouda.000000000310355001.html
Number of Data Point Extracted:  202
https://www.aldi-sued.de/de/p.gut-drei-eichen-pfefferbeisser--g.000000000000203048.html
Number of Data Point Extracted:  203
https://www.aldi-sued.de/de/p.gut-bio-bio-lachsfilet--g.000000000000211484.html
Number of Data Point Extracted:  204
https://www.aldi-sued.de/de/p.gut-drei-eichen-schwarzwaelder-schaeufele--kg.000000000000342279.html
Nu

Number of Data Point Extracted:  261
https://www.aldi-sued.de/de/p.golden-seafood-backfisch-staebchen--g.000000000000198709.html
Number of Data Point Extracted:  262
https://www.aldi-sued.de/de/p.golden-seafood-lachsfilets--g.000000000000191385.html
Number of Data Point Extracted:  263
https://www.aldi-sued.de/de/p.meine-kuchenwelt-hoernchen--g-nuss-nougat.000000000187050002.html
Number of Data Point Extracted:  264
https://www.aldi-sued.de/de/p.meine-kuchenwelt-hoernchen--g-milchcreme.000000000187050001.html
Number of Data Point Extracted:  265
https://www.aldi-sued.de/de/p.iglo-rahmspinat--g.000000000000191916.html
Number of Data Point Extracted:  266
https://www.aldi-sued.de/de/p.jacks-farm-deutsches-brathaehnchen--kg.000000000000191391.html
Number of Data Point Extracted:  267
https://www.aldi-sued.de/de/p.kings-crown-peperoni--g-mild.000000000191634002.html
Number of Data Point Extracted:  268
https://www.aldi-sued.de/de/p.kings-crown-peperoni--g-scharf.000000000191634001.html
Num

Number of Data Point Extracted:  325
https://www.aldi-sued.de/de/p.gut-drei-eichen-belgische-leberpastete--g-schnittlauch.000000000276440003.html
Number of Data Point Extracted:  326
https://www.aldi-sued.de/de/p.hofburger-frischkaesering--g-pfeffer.000000000276008002.html
Number of Data Point Extracted:  327
https://www.aldi-sued.de/de/p.hofburger-frischkaesering--g-schnittlauch.000000000276008005.html
Number of Data Point Extracted:  328
https://www.aldi-sued.de/de/p.gut-drei-eichen-belgische-leberpastete--g-gruener-pfeffer.000000000276440002.html
Number of Data Point Extracted:  329
https://www.aldi-sued.de/de/p.gut-drei-eichen-belgische-leberpastete--g-klassik.000000000276440001.html
Number of Data Point Extracted:  330
https://www.aldi-sued.de/de/p.hofburger-frischkaesering--g-ananas.000000000276008004.html
Number of Data Point Extracted:  331
https://www.aldi-sued.de/de/p.fair--gut-eier-aus-freilandhaltung--stueck.000000000000283182.html
Number of Data Point Extracted:  332
https

Number of Data Point Extracted:  388
https://www.aldi-sued.de/de/p.milsani-activedrink--kg-natur.000000000205540005.html
Number of Data Point Extracted:  389
https://www.aldi-sued.de/de/p.milsani-activedrink--kg-natur---fett.000000000205540006.html
Number of Data Point Extracted:  390
https://www.aldi-sued.de/de/p.milsani-activedrink--kg-multifrucht.000000000205540007.html
Number of Data Point Extracted:  391
https://www.aldi-sued.de/de/p.milsani-activedrink--kg-granatapfel.000000000205540004.html
Number of Data Point Extracted:  392
https://www.aldi-sued.de/de/p.milsani-activedrink--kg-banane-erdbeere.000000000205540003.html
Number of Data Point Extracted:  393
https://www.aldi-sued.de/de/p.bestes-aus-der-region-luftgetrocknete-mettwurst--g.000000000000494258.html
Number of Data Point Extracted:  394
https://www.aldi-sued.de/de/p.nur-nur-natur-bio-frischmilch----l.000000000000493388.html
Number of Data Point Extracted:  395
https://www.aldi-sued.de/de/p.milsani-fettarme-milch----l.490

Number of Data Point Extracted:  453
https://www.aldi-sued.de/de/p.herta-finesse--g-schinken.000000000205207001.html
Number of Data Point Extracted:  454
https://www.aldi-sued.de/de/p.snack-time-wraps--g.490000000000710700.html
Number of Data Point Extracted:  455
https://www.aldi-sued.de/de/p.milsani-griesspudding--g.490000000000001633.html
Number of Data Point Extracted:  456
https://www.aldi-sued.de/de/p.hofburger-schmelzscheiben--g.490000000000003207.html
Number of Data Point Extracted:  457
https://www.aldi-sued.de/de/p.milsani-sahne-kefir-auf-frucht--g.490000000000008241.html
Number of Data Point Extracted:  458
https://www.aldi-sued.de/de/p.milsani-dessert-mit-sahnehaube--g.490000000000007671.html
Number of Data Point Extracted:  459
https://www.aldi-sued.de/de/p.wonnemeyer-brotaufstrich--g.490000000000703283.html
Number of Data Point Extracted:  460
https://www.aldi-sued.de/de/p.wonnemeyer-fleischsalat--g.490000000000008205.html
Number of Data Point Extracted:  461
https://www.

Number of Data Point Extracted:  519
https://www.aldi-sued.de/de/p.wonnemeyer-fruhstucksbackwaren--g.490100000000005795.html
Number of Data Point Extracted:  520
https://www.aldi-sued.de/de/p.gourmet-finest-cuisine-riesengarnelen-mit-schale--g.490200000000811357.html
Number of Data Point Extracted:  521
https://www.aldi-sued.de/de/p.milsani-topi-dessert--g.490000000000051951.html
Number of Data Point Extracted:  522
https://www.aldi-sued.de/de/p.gut-bio-koerniger-bio-frischkaese--g.490000000000702492.html
Number of Data Point Extracted:  523
https://www.aldi-sued.de/de/p.jacks-farm-gaensebrust-mit-knochen--g.490000000000047222.html
Number of Data Point Extracted:  524
https://www.aldi-sued.de/de/p.jacks-farm-chicken-nuggets-mit-dip--g.490000000000052501.html
Number of Data Point Extracted:  525
https://www.aldi-sued.de/de/p.meine-kaesetheke-frischkaesezubereitung--g.490000000000009269.html
Number of Data Point Extracted:  526
https://www.aldi-sued.de/de/p.meine-kuchenwelt-feine-kuchens

Number of Data Point Extracted:  585
https://www.aldi-sued.de/de/p.nur-nur-natur-bio-lyoner-aufschnitt--g-paprika.000000000498372002.html
Number of Data Point Extracted:  586
https://www.aldi-sued.de/de/p.nur-nur-natur-bio-lyoner-aufschnitt--g-klassik.000000000498372001.html
Number of Data Point Extracted:  587
https://www.aldi-sued.de/de/p.bio-wiener-wuerstchen--g.000000000000450268.html
Number of Data Point Extracted:  588
https://www.aldi-sued.de/de/p.nur-nur-natur-bio-bratwurst-schwein--g.000000000498356001.html
Number of Data Point Extracted:  589
https://www.aldi-sued.de/de/p.nur-nur-natur-bio-bratwurst-haehnchen--g.000000000498356002.html
Number of Data Point Extracted:  590
https://www.aldi-sued.de/de/p.gut-bio-bio-haehnchen-spezialitaet--g-klassik.000000000294151002.html
Number of Data Point Extracted:  591
https://www.aldi-sued.de/de/p.gut-bio-bio-haehnchen-spezialitaet--g-paprika.000000000294151001.html
Number of Data Point Extracted:  592
https://www.aldi-sued.de/de/p.milsa